[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/corazzon/finance-data-analysis/blob/main/5.2%20%EC%97%85%EC%A2%85%20%ED%85%8C%EB%A7%88%20%EB%8D%B0%EC%9D%B4%ED%84%B0%20%EC%A0%84%EC%B2%98%EB%A6%AC-input.ipynb)

## 업종/테마 전처리 미션확인

* 팀장님이 우리 회사와 유사한 업종과 테마로 사업을 하고 있는 종목을 수집해서 분석해 달라는 미션을 주셨습니다.
* 코스피에서 해당 업종 혹은 테마의 시가총액이 가장 많은 종목을 찾아주세요.
* 해당 업종/테마의 거래량이 가장 많은 종목을 찾아주세요.
* 최근 52주 최고가에서 현재가 기준으로 가장 손실이 크게 난 종목은 어떤 종목일까요?
* 거래대금이 20위 안에 있는 코스피 종목 중에 "PER"가 가장 낮은 종목은 무엇일까요?


### 주요 업종/테마

* 실습에 참고해 보세요.
* 제약 업종 : https://finance.naver.com/sise/sise_group_detail.nhn?type=upjong&no=35
* 의료기기 : https://finance.naver.com/sise/sise_group_detail.nhn?type=theme&no=175
* 코로나19(덱사메타손) : https://finance.naver.com/sise/sise_group_detail.nhn?type=theme&no=442
* 제약업체 : https://finance.naver.com/sise/sise_group_detail.nhn?type=theme&no=172
* 바이오시밀러(복제 바이오의약품) : https://finance.naver.com/sise/sise_group_detail.nhn?type=theme&no=241

In [1]:
# 라이브러리 로드하기 : 데이터 분석을 위한 pandas, 수치계산을 위한 numpy
import pandas as pd
import numpy as np

In [2]:
# 4.업종/테마주 수집시에 저장해 두었던 csv 파일을 다시 읽어옵니다.
# 'upjong_no_35_연-월-일.csv'
# 종목코드는 꼭 문자형태로 불러옵니다. dtype={"종목코드":np.object}
file_name = "upjong_no_261_23-08-21.csv"
raw = pd.read_csv(file_name,dtype={"종목코드_y":object})
raw = raw.drop('종목코드_x', axis=1)
raw.rename(columns={'종목코드_y':'종목코드'}, inplace=True)
raw.shape

(162, 51)

In [3]:
# 미리보기 합니다.
raw.head()

,종목명,현재가,전일비,등락률,매수호가,매도호가,거래량,거래대금,전일거래량,조회일자,...,동일업종 등락률,종목코드,배당수익률l2022.12,PERlEPS(2022.12),PBRlBPS (2022.12),PERlEPS(2023.06),PBRlBPS (2023.06),PERlEPS(2023.05),PBRlBPS (2023.05),배당수익률l2022.11
0,지놈앤컴퍼니 *,16140.0,2440.0,+17.81%,16440.0,16500.0,917537.0,14801.0,16343.0,23-08-21,...,-0.14%,314130,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,이수앱지스 *,7560.0,750.0,+11.01%,7290.0,7300.0,679560.0,5004.0,272086.0,23-08-21,...,-0.14%,086890,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,카이노스메드 *,4210.0,390.0,+10.21%,4235.0,4245.0,615228.0,2635.0,43792.0,23-08-21,...,-0.14%,284620,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,대봉엘에스 *,11460.0,950.0,+9.04%,11540.0,11590.0,510039.0,5689.0,376893.0,23-08-21,...,-0.21%,078140,0.44%,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,샤페론 *,4520.0,355.0,+8.52%,4510.0,4520.0,848817.0,3778.0,416766.0,23-08-21,...,-0.14%,378800,NaN,N/A l -532원,"3.64배 l 1,239원",NaN,NaN,NaN,NaN,NaN


## 전체 컬럼 출력하기

In [4]:
# 전체 컬럼이 출력되도록 display 설정을 변경합니다.
pd.options.display.max_columns = None

In [5]:
# 전체 컬럼이 보이는지 확인합니다.
raw.head()

,종목명,현재가,전일비,등락률,매수호가,매도호가,거래량,거래대금,전일거래량,조회일자,종목명_전처리,매출액,영업이익,당기순이익,영업이익률,순이익률,ROE(지배주주),부채비율,당좌비율,유보율,EPS(원),PER(배),BPS(원),PBR(배),주당배당금(원),시가배당률(%),배당성향(%),시가총액,시가총액순위,상장주식수,액면가l매매단위,외국인한도주식수(A),외국인보유주식수(B),외국인소진율(B/A),투자의견l목표주가,52주최고l최저,PERlEPS(2023.03),추정PERlEPS,PBRlBPS (2023.03),배당수익률,동일업종 PER,동일업종 등락률,종목코드,배당수익률l2022.12,PERlEPS(2022.12),PBRlBPS (2022.12),PERlEPS(2023.06),PBRlBPS (2023.06),PERlEPS(2023.05),PBRlBPS (2023.05),배당수익률l2022.11
0,지놈앤컴퍼니 *,16140.0,2440.0,+17.81%,16440.0,16500.0,917537.0,14801.0,16343.0,23-08-21,지놈앤컴퍼니,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"2,292 억원",코스닥 369위,14102521,500원 l 1주,14102521,104176,0.74%,N/A l N/A,"32,500 l 12,590","N/A l -3,483원",N/A l N/A,"2.91배 l 5,584원",NaN,88.07배,-0.14%,314130,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,이수앱지스 *,7560.0,750.0,+11.01%,7290.0,7300.0,679560.0,5004.0,272086.0,23-08-21,이수앱지스,171.0,16.0,-5.0,9.36,-2.92,NaN,NaN,NaN,NaN,-15.0,-433.37,NaN,NaN,NaN,NaN,NaN,"2,485 억원",코스닥 335위,33182668,500원 l 1주,33182668,557515,1.68%,N/A l N/A,"8,220 l 4,950",N/A l -198원,N/A l N/A,"4.16배 l 1,800원",NaN,88.07배,-0.14%,086890,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,카이노스메드 *,4210.0,390.0,+10.21%,4235.0,4245.0,615228.0,2635.0,43792.0,23-08-21,카이노스메드,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"1,171 억원",코스닥 732위,27922396,500원 l 1주,27922396,951792,3.41%,N/A l N/A,"7,500 l 3,020",N/A l -592원,N/A l N/A,5.06배 l 829원,NaN,88.07배,-0.14%,284620,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,대봉엘에스 *,11460.0,950.0,+9.04%,11540.0,11590.0,510039.0,5689.0,376893.0,23-08-21,대봉엘에스,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"1,273 억원",코스닥 672위,11086579,500원 l 1주,11086579,203700,1.84%,N/A l N/A,"14,950 l 6,910",16.01배 l 717원,N/A l N/A,"1.06배 l 10,797원",NaN,88.07배,-0.21%,078140,0.44%,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,샤페론 *,4520.0,355.0,+8.52%,4510.0,4520.0,848817.0,3778.0,416766.0,23-08-21,샤페론,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"1,042 억원",코스닥 805위,23071031,500원 l 1주,23071031,86998,0.38%,N/A l N/A,"11,600 l 3,740",NaN,N/A l N/A,NaN,NaN,88.07배,-0.14%,378800,NaN,N/A l -532원,"3.64배 l 1,239원",NaN,NaN,NaN,NaN,NaN


In [6]:
# DataFrame의 column만 따로 봅니다.
raw.columns

Index(['종목명', '현재가', '전일비', '등락률', '매수호가', '매도호가', '거래량', '거래대금', '전일거래량',
       '조회일자', '종목명_전처리', '매출액', '영업이익', '당기순이익', '영업이익률', '순이익률', 'ROE(지배주주)',
       '부채비율', '당좌비율', '유보율', 'EPS(원)', 'PER(배)', 'BPS(원)', 'PBR(배)',
       '주당배당금(원)', '시가배당률(%)', '배당성향(%)', '시가총액', '시가총액순위', '상장주식수',
       '액면가l매매단위', '외국인한도주식수(A)', '외국인보유주식수(B)', '외국인소진율(B/A)', '투자의견l목표주가',
       '52주최고l최저', 'PERlEPS(2023.03)', '추정PERlEPS', 'PBRlBPS (2023.03)',
       '배당수익률', '동일업종 PER', '동일업종 등락률', '종목코드', '배당수익률l2022.12',
       'PERlEPS(2022.12)', 'PBRlBPS (2022.12)', 'PERlEPS(2023.06)',
       'PBRlBPS (2023.06)', 'PERlEPS(2023.05)', 'PBRlBPS (2023.05)',
       '배당수익률l2022.11'],
      dtype='object')

## 결측치

In [7]:
raw.isnull()

,종목명,현재가,전일비,등락률,매수호가,매도호가,거래량,거래대금,전일거래량,조회일자,종목명_전처리,매출액,영업이익,당기순이익,영업이익률,순이익률,ROE(지배주주),부채비율,당좌비율,유보율,EPS(원),PER(배),BPS(원),PBR(배),주당배당금(원),시가배당률(%),배당성향(%),시가총액,시가총액순위,상장주식수,액면가l매매단위,외국인한도주식수(A),외국인보유주식수(B),외국인소진율(B/A),투자의견l목표주가,52주최고l최저,PERlEPS(2023.03),추정PERlEPS,PBRlBPS (2023.03),배당수익률,동일업종 PER,동일업종 등락률,종목코드,배당수익률l2022.12,PERlEPS(2022.12),PBRlBPS (2022.12),PERlEPS(2023.06),PBRlBPS (2023.06),PERlEPS(2023.05),PBRlBPS (2023.05),배당수익률l2022.11
0,False,False,False,False,False,False,False,False,False,False,False,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,True,True,True,True,True,True,True
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,True,True,False,False,True,True,True,True,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,True,True,True,True,True,True,True
2,False,False,False,False,False,False,False,False,False,False,False,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,True,True,True,True,True,True,True
3,False,False,False,False,False,False,False,False,False,False,False,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,True,True,True,True,True,True
4,False,False,False,False,False,False,False,False,False,False,False,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,False,False,False,False,False,False,False,False,False,True,False,True,True,False,False,False,True,False,False,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,True,True,False,False,True,True,True,True,True,False,False,False,False,False,False,False,False,False,True,False,True,True,False,False,False,False,True,True,False,False,True,True,True
158,False,False,False,False,False,False,False,False,False,False,False,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,True,True,True,True,True,True,True
159,False,False,False,False,False,False,False,False,False,False,False,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,True,True,True,True,True,True,True
160,False,False,False,False,False,False,False,False,False,False,False,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,True,True,True,True,True,True,True


In [8]:
# 결측치의 합계를 구합니다.
raw.isnull().sum()

종목명                    0
현재가                    0
전일비                    0
등락률                    0
매수호가                   0
매도호가                   0
거래량                    0
거래대금                   0
전일거래량                  0
조회일자                   0
종목명_전처리                0
매출액                  128
영업이익                 129
당기순이익                135
영업이익률                129
순이익률                 136
ROE(지배주주)            162
부채비율                 162
당좌비율                 162
유보율                  162
EPS(원)               144
PER(배)               144
BPS(원)               162
PBR(배)               162
주당배당금(원)             162
시가배당률(%)             162
배당성향(%)              162
시가총액                   0
시가총액순위                 0
상장주식수                  0
액면가l매매단위               0
외국인한도주식수(A)            0
외국인보유주식수(B)            0
외국인소진율(B/A)            0
투자의견l목표주가              0
52주최고l최저               0
PERlEPS(2023.03)      48
추정PERlEPS              0
PBRlBPS (2023.03)     48
배당수익률                162


In [9]:
# 결측치의 비율을 구합니다.
raw.isnull().mean() * 100

종목명                    0.000000
현재가                    0.000000
전일비                    0.000000
등락률                    0.000000
매수호가                   0.000000
매도호가                   0.000000
거래량                    0.000000
거래대금                   0.000000
전일거래량                  0.000000
조회일자                   0.000000
종목명_전처리                0.000000
매출액                   79.012346
영업이익                  79.629630
당기순이익                 83.333333
영업이익률                 79.629630
순이익률                  83.950617
ROE(지배주주)            100.000000
부채비율                 100.000000
당좌비율                 100.000000
유보율                  100.000000
EPS(원)                88.888889
PER(배)                88.888889
BPS(원)               100.000000
PBR(배)               100.000000
주당배당금(원)             100.000000
시가배당률(%)             100.000000
배당성향(%)              100.000000
시가총액                   0.000000
시가총액순위                 0.000000
상장주식수                  0.000000
액면가l매매단위               0.000000
외국인한도주식수

In [10]:
# 결측치 제거 전에 행과 열의 수를 봅니다.
raw.shape

(162, 51)

In [11]:
# 전체가 결측치인 컬럼은 삭제 합니다.
df = raw.dropna(axis=1, how='all').copy()
df

,종목명,현재가,전일비,등락률,매수호가,매도호가,거래량,거래대금,전일거래량,조회일자,종목명_전처리,매출액,영업이익,당기순이익,영업이익률,순이익률,EPS(원),PER(배),시가총액,시가총액순위,상장주식수,액면가l매매단위,외국인한도주식수(A),외국인보유주식수(B),외국인소진율(B/A),투자의견l목표주가,52주최고l최저,PERlEPS(2023.03),추정PERlEPS,PBRlBPS (2023.03),동일업종 PER,동일업종 등락률,종목코드,배당수익률l2022.12,PERlEPS(2022.12),PBRlBPS (2022.12),PERlEPS(2023.06),PBRlBPS (2023.06),PERlEPS(2023.05),PBRlBPS (2023.05),배당수익률l2022.11
0,지놈앤컴퍼니 *,16140.0,2440.0,+17.81%,16440.0,16500.0,917537.0,14801.0,16343.0,23-08-21,지놈앤컴퍼니,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"2,292 억원",코스닥 369위,14102521,500원 l 1주,14102521,104176,0.74%,N/A l N/A,"32,500 l 12,590","N/A l -3,483원",N/A l N/A,"2.91배 l 5,584원",88.07배,-0.14%,314130,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,이수앱지스 *,7560.0,750.0,+11.01%,7290.0,7300.0,679560.0,5004.0,272086.0,23-08-21,이수앱지스,171.0,16.0,-5.0,9.36,-2.92,-15.0,-433.37,"2,485 억원",코스닥 335위,33182668,500원 l 1주,33182668,557515,1.68%,N/A l N/A,"8,220 l 4,950",N/A l -198원,N/A l N/A,"4.16배 l 1,800원",88.07배,-0.14%,086890,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,카이노스메드 *,4210.0,390.0,+10.21%,4235.0,4245.0,615228.0,2635.0,43792.0,23-08-21,카이노스메드,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"1,171 억원",코스닥 732위,27922396,500원 l 1주,27922396,951792,3.41%,N/A l N/A,"7,500 l 3,020",N/A l -592원,N/A l N/A,5.06배 l 829원,88.07배,-0.14%,284620,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,대봉엘에스 *,11460.0,950.0,+9.04%,11540.0,11590.0,510039.0,5689.0,376893.0,23-08-21,대봉엘에스,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"1,273 억원",코스닥 672위,11086579,500원 l 1주,11086579,203700,1.84%,N/A l N/A,"14,950 l 6,910",16.01배 l 717원,N/A l N/A,"1.06배 l 10,797원",88.07배,-0.21%,078140,0.44%,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,샤페론 *,4520.0,355.0,+8.52%,4510.0,4520.0,848817.0,3778.0,416766.0,23-08-21,샤페론,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"1,042 억원",코스닥 805위,23071031,500원 l 1주,23071031,86998,0.38%,N/A l N/A,"11,600 l 3,740",NaN,N/A l N/A,NaN,88.07배,-0.14%,378800,NaN,N/A l -532원,"3.64배 l 1,239원",NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,코스맥스비티아이,10010.0,170.0,-1.67%,9930.0,9940.0,11492.0,114.0,17359.0,23-08-21,코스맥스비티아이,6291.0,2101.0,1580.0,33.40,25.11,1153.0,130.49,"21조 2,577 억원",코스피 15위,146402770,"1,000원 l 1주",146402770,29589485,20.21%,"4.00매수 l 211,250","196,508 l 139,500",NaN,"34.00배 l 4,279원",NaN,88.07배,-0.08%,068270,0.25%,NaN,NaN,"38.72배 l 3,750원","4.85배 l 29,918원",NaN,NaN,NaN
158,종근당바이오,22550.0,400.0,-1.74%,22650.0,22700.0,3368.0,76.0,6088.0,23-08-21,종근당바이오,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"2,734 억원",코스닥 307위,22102155,500원 l 1주,22102155,640284,2.90%,N/A l N/A,"14,970 l 6,120",N/A l -591원,N/A l N/A,"3.95배 l 3,135원",88.07배,-0.08%,220100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
159,환인제약,14710.0,270.0,-1.80%,14690.0,14700.0,355234.0,5454.0,66708.0,23-08-21,환인제약,NaN,NaN,NaN,NaN,NaN,NaN,NaN,946 억원,코스닥 889위,51881094,500원 l 1주,51881094,0,0.00%,N/A l N/A,"2,495 l 235",5.58배 l 327원,N/A l N/A,"0.30배 l 6,064원",88.07배,-0.14%,109960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
160,에이프릴바이오 *,22350.0,450.0,-1.97%,21950.0,22050.0,214491.0,4817.0,330266.0,23-08-21,에이프릴바이오,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"8,993 억원",코스닥 70위,12647869,500원 l 1주,12647869,235547,1.86%,"4.00매수 l 97,715","90,000 l 19,297","N/A l -1,746원",N/A l N/A,"33.71배 l 2,109원",88.07배,-0.14%,310210,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# 삭제된 컬럼의 수를 계산해 봅니다.
raw.shape[1] - df.shape[1]

10

In [13]:
# info를 통해 결측치 데이터 타입을 확인합니다.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162 entries, 0 to 161
Data columns (total 41 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   종목명                162 non-null    object 
 1   현재가                162 non-null    float64
 2   전일비                162 non-null    float64
 3   등락률                162 non-null    object 
 4   매수호가               162 non-null    float64
 5   매도호가               162 non-null    float64
 6   거래량                162 non-null    float64
 7   거래대금               162 non-null    float64
 8   전일거래량              162 non-null    float64
 9   조회일자               162 non-null    object 
 10  종목명_전처리            162 non-null    object 
 11  매출액                34 non-null     float64
 12  영업이익               33 non-null     float64
 13  당기순이익              27 non-null     object 
 14  영업이익률              33 non-null     float64
 15  순이익률               26 non-null     float64
 16  EPS(원)             18 non-

## 파생변수 만들기와 텍스트 데이터 다루기
<img src="https://pandas.pydata.org/docs/_images/05_newcolumn_1.svg">

* 코스피/코스닥 구분 만들기
* 시가총액 순위 분리하기
* 액면가l매매단위
* 52주최고l최저

### 코스피/코스닥 컬럼 만들기

In [14]:
# "코스피/코스닥" 컬럼을 만듭니다.
#  * 는 코스닥 종목입니다. str.endswith 를 사용합니다.
df.loc[df['종목명'].str.endswith('*'), "코스피/코스닥"] = '코스닥'
df.loc[~df['종목명'].str.endswith('*'), "코스피/코스닥"] = '코스피'

In [15]:
df['코스피/코스닥'].tail()

157    코스피
158    코스피
159    코스피
160    코스닥
161    코스닥
Name: 코스피/코스닥, dtype: object

In [16]:
# "코스피/코스닥" 빈도수를 확인합니다.
df['코스피/코스닥'].value_counts()

코스닥    92
코스피    70
Name: 코스피/코스닥, dtype: int64

###  시가총액 수치형태로 만들기

In [17]:
# or를 의미하는 | 는 엔터키 위에 있는 키를 shift키를 누르면 나옵니다.
# df['시가총액(억원)'] = df['시가총액'].str.replace("억원","")
# df['시가총액(억원)'] = df['시가총액(억원)'].str.replace("조|,", "")
# df['시가총액(억원)'] = df['시가총액(억원)'].str.replace(" ", "")
# df['시가총액(억원)'] = df['시가총액(억원)'].str.strip()
# df['시가총액(억원)'] = df['시가총액(억원)'].astype(int)
# df['시가총액(억원)']

* [정규 표현식 - 위키백과, 우리 모두의 백과사전](https://ko.wikipedia.org/wiki/%EC%A0%95%EA%B7%9C_%ED%91%9C%ED%98%84%EC%8B%9D)

In [18]:
# 간단하게 바꿀 수 있는 방법
# 정규표현식을 사용한 숫자 추출
df['시가총액(억원)'] = df['시가총액'].str.replace("[^0-9]","").astype(int)

C:\Users\petpulslab\AppData\Local\Temp\ipykernel_26988\1790135301.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df['시가총액(억원)'] = df['시가총액'].str.replace("[^0-9]","").astype(int)


### 시가총액 순위 수치데이터로 만들기

In [19]:
# split을 사용하는 방법
# '시가총액순위(전처리)' column에서도 '위'를 빼고 수치데이터로 변경합니다.
# df["시가총액순위(전처리)"] = df['시가총액순위'].str.split(" ", expand=True)[1].str.replace("[^0-9]","").astype(int)
# df["시가총액순위(전처리)"]

In [20]:
# 정규표현식을 사용하는 방법도 있습니다. 
# 숫자를 제외한 문자는 빈문자""로 변경하도록 합니다.
df["시가총액순위(전처리)"] = df['시가총액순위'].str.replace("[^0-9]","").astype(int)
df["시가총액순위(전처리)"]

C:\Users\petpulslab\AppData\Local\Temp\ipykernel_26988\4272781163.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df["시가총액순위(전처리)"] = df['시가총액순위'].str.replace("[^0-9]","").astype(int)


0      369
1      335
2      732
3      672
4      805
      ... 
157     15
158    307
159    889
160     70
161    817
Name: 시가총액순위(전처리), Length: 162, dtype: int32

### 52주최고l최저 

In [21]:
df['52주최고l최저']

0        32,500  l  12,590
1          8,220  l  4,950
2          7,500  l  3,020
3         14,950  l  6,910
4         11,600  l  3,740
              ...         
157    196,508  l  139,500
158       14,970  l  6,120
159          2,495  l  235
160      90,000  l  19,297
161       13,000  l  4,150
Name: 52주최고l최저, Length: 162, dtype: object

In [22]:
# '52주최고l최저' => 52주최고, 52주최저 두 개의  컬럼으로 나누어 전처리 합니다.
df["52주최고"] = df['52주최고l최저'].str.split('  l  ', expand=True)[0]
df["52주최저"] = df['52주최고l최저'].str.split('  l  ', expand=True)[1]
df.loc[:,['52주최고','52주최저']]

,52주최고,52주최저
0,"32,500","12,590"
1,"8,220","4,950"
2,"7,500","3,020"
3,"14,950","6,910"
4,"11,600","3,740"
...,...,...
157,"196,508","139,500"
158,"14,970","6,120"
159,"2,495",235
160,"90,000","19,297"


In [23]:
# 일부 column들을 수치형 데이터로 바꿀수 있도록 ','와 같은 문자들을 없애고
# 새로운 column에 수치형 데이터로 입력합니다.
# 이런 수치형 데이터의 column들로 새로운 데이터 column들을 생성합니다.
df['52주최고'] = df['52주최고'].str.replace(",","").astype(int)
df['52주최저'] = df['52주최저'].str.replace(",","").astype(int)

In [24]:
# 52주차액: 52주최고 - 52주최저 
# 52주차액비율 : 52주최고 / 52주최저 
df["52주차액"] = df['52주최고'] - df['52주최저']
df["52주차액비율"] = df['52주최고'] / df['52주최저'] 


In [25]:
# 현재-최고가 : 현재가 - 52주최고
# 현재/최고가비율 : 현재가 / 52주최고
df["현재-최고가"] = df['현재가'] - df['52주최고']
df["현재/최고가비율"] = df['현재가'] / df['52주최고']


In [26]:
df[["종목명", "52주차액", "52주차액비율", "52주최고", "52주최저", "현재-최고가", "현재/최고가비율"]].head()

,종목명,52주차액,52주차액비율,52주최고,52주최저,현재-최고가,현재/최고가비율
0,지놈앤컴퍼니 *,19910,2.581414,32500,12590,-16360.0,0.496615
1,이수앱지스 *,3270,1.660606,8220,4950,-660.0,0.919708
2,카이노스메드 *,4480,2.483444,7500,3020,-3290.0,0.561333
3,대봉엘에스 *,8040,2.163531,14950,6910,-3490.0,0.766555
4,샤페론 *,7860,3.101604,11600,3740,-7080.0,0.389655


### 투자의견l목표주가 astype과 to_numeric

In [27]:
df["투자의견l목표주가"]

0             N/A  l  N/A
1             N/A  l  N/A
2             N/A  l  N/A
3             N/A  l  N/A
4             N/A  l  N/A
              ...        
157    4.00매수  l  211,250
158           N/A  l  N/A
159           N/A  l  N/A
160     4.00매수  l  97,715
161           N/A  l  N/A
Name: 투자의견l목표주가, Length: 162, dtype: object

In [28]:
# '투자의견l목표주가' => 투자의견, 매수매도, 목표주가 컬럼으로 나누어 전처리 합니다.

df["투자의견"] = df["투자의견l목표주가"].str.split('  l  ', expand = True)[0]
df["목표주가"] = df["투자의견l목표주가"].str.split('  l  ', expand = True)[1]
df["목표주가"] = df["목표주가"].replace("N/A", np.nan).str.replace(",", "").astype(float)
df['목표주가']

0           NaN
1           NaN
2           NaN
3           NaN
4           NaN
         ...   
157    211250.0
158         NaN
159         NaN
160     97715.0
161         NaN
Name: 목표주가, Length: 162, dtype: float64

In [29]:
df['투자의견'] = df['투자의견'].replace("N/A", np.nan)
df["매수매도"] = df["투자의견"].str.replace("[0-9.]","")
df['투자의견'] = df['투자의견'].str.replace("[^0-9.]","").astype(float)
df[['투자의견','매수매도']]

C:\Users\petpulslab\AppData\Local\Temp\ipykernel_26988\3217570349.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df["매수매도"] = df["투자의견"].str.replace("[0-9.]","")
C:\Users\petpulslab\AppData\Local\Temp\ipykernel_26988\3217570349.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df['투자의견'] = df['투자의견'].str.replace("[^0-9.]","").astype(float)


,투자의견,매수매도
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
...,...,...
157,4.0,매수
158,NaN,NaN
159,NaN,NaN
160,4.0,매수


In [30]:
# df["투자의견"].astype(float)

In [31]:
# 투자의견의 수치만 pd.to_numeric 으로 숫자로 만들기
# 강제로 숫자 타입으로 변경하기
# df['투자의견'] = pd.to_numeric(df['투자의견'])

In [32]:
# 투자의견 컬럼만 가져와서 describe로 기술통계 값을 봅니다.
df['투자의견'].describe()

count    37.000000
mean      3.977568
std       0.136451
min       3.170000
25%       4.000000
50%       4.000000
75%       4.000000
max       4.000000
Name: 투자의견, dtype: float64

### 데이터 타입 변경하기
* "동일업종 PER 컬럼"에서 "배" 문자를 제거하고 수치 변수로 변환해 주세요.
* 전처리 결괏값은 df["동일업종 PER(배)"]라는 컬럼에 담아주세요

In [33]:
# "동일업종 PER"
df['동일업종 PER']

0      88.07배
1      88.07배
2      88.07배
3      88.07배
4      88.07배
        ...  
157    88.07배
158    88.07배
159    88.07배
160    88.07배
161    88.07배
Name: 동일업종 PER, Length: 162, dtype: object

In [34]:
# "동일업종 PER 컬럼"에서 "배" 문자를 제거하고 수치 변수로 변환해 주세요.
df['동일업종 PER'] = df['동일업종 PER'].str.replace("배","")

In [35]:
# str.replace, astype를 이용하면 쉽게 해결할 수 있습니다.
df['동일업종 PER'] = df['동일업종 PER'].astype(float)
df['동일업종 PER']

0      88.07
1      88.07
2      88.07
3      88.07
4      88.07
       ...  
157    88.07
158    88.07
159    88.07
160    88.07
161    88.07
Name: 동일업종 PER, Length: 162, dtype: float64

## filter와 melt로 여러 컬럼 전처리하기

### PER와 EPS 

#### PER 주가수익비율

* "이 주식은 비싼가, 주식시장이 거품인가" 판단하는 시금석
* PER = 주식가격 / 주당순이익(EPS)

물건을 살 때 값을 얼마나 줘야 좋은 건지 판단할 기준은 수없이 많다. 내게 얼마나 절실한가. 이 물건이 얼마나 유용하고 값어치가 있나. 혹은 어느 정도나 오래 사용할 수 있나. 물건에 대한 다른 사람들의 평판은 어떤가. 물건이 내게 주는 수익은 어느 정도인가…. 생각해보면 쉽게 떠오르는 이런저런 이유들은 어려운 주식투자나, 투자결정에도 그대로 적용된다.

이런 점에서 흔히 투자자들 사이에서 `퍼(PER)`로 불리는 **주가수익비율(Price earning ratio, 이하 PER)**은 어떤 회사의 주식가치, 더 나아가 전체 주식시장의 가치가 고평가 됐는지 가늠할 수 있는 유용한 잣대다. **PER은 현재 시장에서 매매되는 특정회사의 주식가격을 주당순이익으로 나눈 값을 말한다.** 한 주에 만 원하는 회사주식이 1년에 주당 1000원의 순이익을 낸다면? PER은 10이 된다. 이 수치가 회사의 가치를 어떻게 반영한다는 얘기일까?

* 출처 [네이버 지식백과]주가수익비율(PER) - "이 주식은 비싼가, 주식시장이 거품인가" 판단하는 시금석 (금융사전, 김태근)
* https://terms.naver.com/entry.nhn?docId=3571127&cid=58781&categoryId=58781

#### EPS 주당순이익

* EPS =  순이익(당기순이익) / 그 기업이 발행한 총 주식수

주당순이익(EPS)은 기업이 벌어들인 순이익(당기순이익)을 그 기업이 발행한 총 주식수로 나눈 값으로, 1주당 이익을 얼마나 창출하였느냐를 나타내는 지표이다. 즉 해당 회사가 1년간 올린 수익에 대한 주주의 몫을 나타내는 지표라 할 수 있다. 또한 주당순이익은 주가수익비율(PER)계산의 기초가 되기도 한다.

EPS가 높을수록 주식의 투자 가치는 높다고 볼 수 있으며, 그만큼 해당 회사의 경영실적이 양호하다는 뜻이다. 따라서 배당 여력도 많으므로 주가에 긍정적인 영향을 미친다.

EPS는 당기순이익 규모가 늘면 높아지게 되고, 전환사채의 주식전환이나 증자로 주식수가 많아지면 낮아지게 된다. 특히 최근 주식시장의 패턴이 기업의 수익성을 중시하는 쪽으로 바뀌면서 EPS의 크기가 중요시되고 있다.

* 출처 : [네이버 지식백과] 주당순이익(EPS) (시사상식사전, pmg 지식엔진연구소)
* https://terms.naver.com/entry.nhn?docId=74510&cid=43667&categoryId=43667

### filter 를 사용해 특정 텍스트가 들어가는 컬럼 찾기
* [정규 표현식 - 위키백과, 우리 모두의 백과사전](https://ko.wikipedia.org/wiki/%EC%A0%95%EA%B7%9C_%ED%91%9C%ED%98%84%EC%8B%9D)

In [36]:
# df.filter(regex=)를 통해 "PERlEPS" 가 들어가는 컬럼만 가져옵니다.
# 특정 컬럼을 가져오고 종목코드를 추가해 줍니다.
df_per_eps = df.filter(regex="^PERlEPS").copy()
df_per_eps['종목코드'] = df['종목코드']
df_per_eps

,PERlEPS(2023.03),PERlEPS(2022.12),PERlEPS(2023.06),PERlEPS(2023.05),종목코드
0,"N/A l -3,483원",NaN,NaN,NaN,314130
1,N/A l -198원,NaN,NaN,NaN,086890
2,N/A l -592원,NaN,NaN,NaN,284620
3,16.01배 l 717원,NaN,NaN,NaN,078140
4,NaN,N/A l -532원,NaN,NaN,378800
...,...,...,...,...,...
157,NaN,NaN,"38.72배 l 3,750원",NaN,068270
158,N/A l -591원,NaN,NaN,NaN,220100
159,5.58배 l 327원,NaN,NaN,NaN,109960
160,"N/A l -1,746원",NaN,NaN,NaN,310210


### melt로 tidy data 만들기

<img src="https://pandas.pydata.org/docs/_images/reshaping_melt.png" width="600">

* https://pandas.pydata.org/docs/user_guide/reshaping.html#reshaping-by-melt
* [Tidy Data 란?](https://vita.had.co.nz/papers/tidy-data.pdf)


In [37]:
# melt를 통해 종목코드를 녹여냅니다.
df_per_eps_melt = df_per_eps.melt(id_vars='종목코드')

In [38]:
# 종목코드별로 value_counts를 통해  PERlEPS 행이 몇 개인지 봅니다.
df_per_eps_melt['종목코드'].value_counts()

314130    4
053030    4
106190    4
183490    4
187660    4
         ..
009290    4
003090    4
234690    4
036480    4
288330    4
Name: 종목코드, Length: 162, dtype: int64

### 결측치 데이터 제거
<img src="https://pandas.pydata.org/docs/_images/03_subset_rows.svg">

In [39]:
# PERlEPS 데이터프레임에서 결측치 데이터 제거
# 여기에서 기간별로 생긴 행은 이전 실습에서 concat으로 데이터를 합칠 때 
# 컬럼의 이름이 다르면 새로운 컬럼으로 생성하기 때문입니다.
df_pe = df_per_eps_melt.dropna()
df_pe

,종목코드,variable,value
0,314130,PERlEPS(2023.03),"N/A l -3,483원"
1,086890,PERlEPS(2023.03),N/A l -198원
2,284620,PERlEPS(2023.03),N/A l -592원
3,078140,PERlEPS(2023.03),16.01배 l 717원
5,114450,PERlEPS(2023.03),N/A l -301원
...,...,...,...
470,016580,PERlEPS(2023.06),"10.55배 l 1,392원"
477,223250,PERlEPS(2023.06),19.68배 l 693원
479,000227,PERlEPS(2023.06),N/A l -386원
481,068270,PERlEPS(2023.06),"38.72배 l 3,750원"


In [40]:
# variable 컬럼명을 rename을 통해 "PERlEPS"로 변경합니다.
df_pe = df_pe.rename(columns = {'variable':'PERlEPS'})
df_pe

,종목코드,PERlEPS,value
0,314130,PERlEPS(2023.03),"N/A l -3,483원"
1,086890,PERlEPS(2023.03),N/A l -198원
2,284620,PERlEPS(2023.03),N/A l -592원
3,078140,PERlEPS(2023.03),16.01배 l 717원
5,114450,PERlEPS(2023.03),N/A l -301원
...,...,...,...
470,016580,PERlEPS(2023.06),"10.55배 l 1,392원"
477,223250,PERlEPS(2023.06),19.68배 l 693원
479,000227,PERlEPS(2023.06),N/A l -386원
481,068270,PERlEPS(2023.06),"38.72배 l 3,750원"


In [41]:
# df_pe에서 PER, EPS 컬럼을 따로 생성합니다.
df_pe["PER"] = df_pe['value'].str.split('  l  ', expand = True)[0]
df_pe["EPS"] = df_pe['value'].str.split('  l  ', expand = True)[1]
df_pe

,종목코드,PERlEPS,value,PER,EPS
0,314130,PERlEPS(2023.03),"N/A l -3,483원",N/A,"-3,483원"
1,086890,PERlEPS(2023.03),N/A l -198원,N/A,-198원
2,284620,PERlEPS(2023.03),N/A l -592원,N/A,-592원
3,078140,PERlEPS(2023.03),16.01배 l 717원,16.01배,717원
5,114450,PERlEPS(2023.03),N/A l -301원,N/A,-301원
...,...,...,...,...,...
470,016580,PERlEPS(2023.06),"10.55배 l 1,392원",10.55배,"1,392원"
477,223250,PERlEPS(2023.06),19.68배 l 693원,19.68배,693원
479,000227,PERlEPS(2023.06),N/A l -386원,N/A,-386원
481,068270,PERlEPS(2023.06),"38.72배 l 3,750원",38.72배,"3,750원"


In [42]:
# PER, EPS 컬럼의 N/A 값을 결측치로 만듭니다.
df_pe['PER'] = df_pe['PER'].replace("N/A", np.nan)
df_pe['EPS'] = df_pe['EPS'].replace("N/A", np.nan)

In [43]:
#  PER, EPS 값 중 둘 다 결측치는 notnull 혹은 isnull을 사용해 제거합니다.
df_pe = df_pe[df_pe['PER'].notnull() | df_pe['EPS'].notnull()].copy()

In [44]:
# PER(배) 컬럼을 만들고 수치데이터 형태로 변환합니다.
df_pe["PER(배)"] = df_pe['PER'].str.replace("[^0-9.]","").astype(float)

C:\Users\petpulslab\AppData\Local\Temp\ipykernel_26988\884048284.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df_pe["PER(배)"] = df_pe['PER'].str.replace("[^0-9.]","").astype(float)


In [45]:
# EPS(원) 컬럼을 만들고 수치데이터 형태로 변환합니다.
df_pe["EPS(원)"] = df_pe['EPS'].str.replace("원|,","").astype(int)

C:\Users\petpulslab\AppData\Local\Temp\ipykernel_26988\3960350041.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df_pe["EPS(원)"] = df_pe['EPS'].str.replace("원|,","").astype(int)


In [46]:
# 필요한 컬럼만 따로 가져옵니다.
df_pe = df_pe[['종목코드','PERlEPS','PER(배)', 'EPS(원)']].copy()
df_pe

,종목코드,PERlEPS,PER(배),EPS(원)
0,314130,PERlEPS(2023.03),NaN,-3483
1,086890,PERlEPS(2023.03),NaN,-198
2,284620,PERlEPS(2023.03),NaN,-592
3,078140,PERlEPS(2023.03),16.01,717
5,114450,PERlEPS(2023.03),NaN,-301
...,...,...,...,...
470,016580,PERlEPS(2023.06),10.55,1392
477,223250,PERlEPS(2023.06),19.68,693
479,000227,PERlEPS(2023.06),NaN,-386
481,068270,PERlEPS(2023.06),38.72,3750


### 중복 데이터 확인

In [47]:
# "종목코드"기준으로 중복된 데이터가 있는지 찾습니다.
df_pe[df_pe.duplicated("종목코드")]

,종목코드,PERlEPS,PER(배),EPS(원)


In [48]:
# 만약 중복데이터가 있다면 다음과 같은 방법으로 제거합니다.
# 추출했던 원래 데이터프레임 df와 다시 merge 로 합치기 위해 
# drop_duplicates 를 사용해 종목코드가 중복되는 행은 제거합니다.
# "종목코드", "PERlEPS" 로 정렬을 하고 
# drop_duplicates 로 제거시 keep="first" 로 위에 있는 데이터를 남깁니다.


# df_pe = df_pe.sort_values(["종목코드", "PERlEPS"])
# df_pe = df_pe.drop_duplicates(["종목코드", "PERlEPS"])

In [49]:
# 종목코드값 기준으로 중복데이터를 확인합니다.
df_pe['종목코드'].value_counts().unique()

array([1], dtype=int64)

### PBRlBPS 전처리


#### PBR(주가순자산비율, price-to-book ratio, P/B ratio)

* PBR = 주가/BPS(주당순자산가치)
* 주가는 그 회사의 종합적인 평가이므로 주주 소유분을 초과한 부분은 모두 그 회사의 잠재적인 프리미엄이 되기 때문에 **경영의 종합력이 뛰어나면 뛰어날수록 배율이 높아진다**고 할 수 있다. 일반적으로 이 지표는 PER과 함께 사용되는데, 이는 주가순자산배율이 그 회사를 정적인 상태 즉 스토크(stock)면에서 보고 있는데 반하여 주가수익률은 동적인 상태 즉 플로(flow)면에서 보고 있으므로 두 가지 지표가 서로 보완관계에 있다.

* 다만 이 주가 순자산배율에도 **주주소유분이 장부가격에 의해 계산되고 있어 반드시 정확하다고 할 수 없고 또 그 계산기준이 전기의 결산기가 된다는 등의 난점이 있다.**
* 출처 : [주가순자산비율 - 위키백과, 우리 모두의 백과사전](https://ko.wikipedia.org/wiki/%EC%A3%BC%EA%B0%80%EC%88%9C%EC%9E%90%EC%82%B0%EB%B9%84%EC%9C%A8)

#### BPS(주당순자산가치, Book-value Per Share)

* 주당순자산가치 = 기업의 순자산/ 발행 주식수
* 주당순자산가치는, '청산가치' 라고도 불린다. 그 이유는 현재 시점에서 기업의 활동을 중단시키고 그 부를 모든 주주들에게 나눠줄 경우 한주당 얼마씩이 돌아가는가를 나타내는 수치이기 때문이다. 보통 주당순자산가치라는 말은 잘 쓰지 않고 BPS라고 하므로 이하 BPS로 쓴다.

* BPS가 높다는 것은 자기자본의 비중이 크거나 회사 가치가 높다는 것을 나타낸다. BPS에는 주가 정보가 고려되어 있지 않으므로, 이 회사의 주가가 자산 가치에 비해 얼마나 저평가 혹은 고평가되어 있는지 판단하기 위해 PBR이라는 값을 사용한다.

* PBR 값은 BPS 값으로 현재 주가를 나눈 것이다. BPS가 1만원인 위 사례에서 주가가 5,000원이라면 PBR은 0.5가 되고, 주가가 20,000원이라면 PBR은 2가 된다. 즉 전자의 경우 주가는 실제의 자산에 비해 저평가되어 되어 있는 것이고, 후자의 경우 주가는 실제 자산에 비해 고평가 되어 있는 것이다.

* 출처 : [주당순자산가치 - 위키백과, 우리 모두의 백과사전](https://ko.wikipedia.org/wiki/%EC%A3%BC%EB%8B%B9%EC%88%9C%EC%9E%90%EC%82%B0%EA%B0%80%EC%B9%98)


In [53]:
# filter를 사용해 PBRlBPS 컬럼 가져옵니다.
# 종목코드 추가합니다.
df_pbr_bps = df.filter(regex="PBRlBPS").copy()
df_pbr_bps['종목코드'] = df['종목코드']
df_pbr_bps

,PBRlBPS (2023.03),PBRlBPS (2022.12),PBRlBPS (2023.06),PBRlBPS (2023.05),종목코드
0,"2.91배 l 5,584원",NaN,NaN,NaN,314130
1,"4.16배 l 1,800원",NaN,NaN,NaN,086890
2,5.06배 l 829원,NaN,NaN,NaN,284620
3,"1.06배 l 10,797원",NaN,NaN,NaN,078140
4,NaN,"3.64배 l 1,239원",NaN,NaN,378800
...,...,...,...,...,...
157,NaN,NaN,"4.85배 l 29,918원",NaN,068270
158,"3.95배 l 3,135원",NaN,NaN,NaN,220100
159,"0.30배 l 6,064원",NaN,NaN,NaN,109960
160,"33.71배 l 2,109원",NaN,NaN,NaN,310210


In [57]:
# melt로 tidy data 만듭니다.
df_pbr_bps_melt = df_pbr_bps.melt(id_vars='종목코드').copy()

In [59]:
# 결측치를 제거합니다.
df_pb = df_pbr_bps_melt.dropna()

In [61]:
# variable 컬럼명을 "PBRlBPS"로 변경합니다.
df_pb = df_pb.rename(columns = {'variable' : 'PBRlBPS'})
df_pb.head()

,종목코드,PBRlBPS,value
0,314130,PBRlBPS (2023.03),"2.91배 l 5,584원"
1,086890,PBRlBPS (2023.03),"4.16배 l 1,800원"
2,284620,PBRlBPS (2023.03),5.06배 l 829원
3,078140,PBRlBPS (2023.03),"1.06배 l 10,797원"
5,114450,PBRlBPS (2023.03),"0.92배 l 3,452원"


In [64]:
# PBR, BPS 컬럼을 만듭니다.
df_pb['PBR'] = df_pb['value'].str.split('  l  ', expand = True)[0]
df_pb['BPS'] = df_pb['value'].str.split('  l  ', expand = True)[1]

In [66]:
# N/A 값(문자) np.nan(결측치)로 변환합니다.
df_pb['PBR'] = df_pb['PBR'].replace("N/A",np.nan)
df_pb['BPS'] = df_pb['BPS'].replace("N/A",np.nan)

In [71]:
# PBR(배), BPS(원) 컬럼을 만듭니다.
df_pb["PBR(배)"] = df_pb["PBR"].str.replace('배',"").astype(float)
df_pb["BPS(원)"] = df_pb["BPS"].str.replace('원|,',"").astype(float)

C:\Users\petpulslab\AppData\Local\Temp\ipykernel_26988\641810961.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df_pb["BPS(원)"] = df_pb["BPS"].str.replace('원|,',"").astype(float)


In [76]:
# 종목코드 기준 value_counts 를 구해 중복값이 있는지 확인합니다.
df_pb['종목코드'].value_counts().unique()

array([1], dtype=int64)

In [80]:
# "종목코드","PBRlBPS" 로 중복값이 있는지 확인합니다.
# 만약 중복값이 있다면 drop_duplicates 로 삭제합니다.
df_pb[df_pb.duplicated(['종목코드'])]

,종목코드,PBRlBPS,PBR(배),BPS(원)


In [81]:
# 사용할 컬럼만 따로 지정해줍니다.
# "종목코드", "PBRlBPS", "PBR(배)", "BPS(원)"
df_pb = df_pb[["종목코드", "PBRlBPS", "PBR(배)", "BPS(원)"]].copy()
df_pb.head()

,종목코드,PBRlBPS,PBR(배),BPS(원)
0,314130,PBRlBPS (2023.03),2.91,5584.0
1,086890,PBRlBPS (2023.03),4.16,1800.0
2,284620,PBRlBPS (2023.03),5.06,829.0
3,078140,PBRlBPS (2023.03),1.06,10797.0
5,114450,PBRlBPS (2023.03),0.92,3452.0


### 배당 수익률 전처리

In [ ]:
# PER|EPS 전처리와 마찬가리로 배당수익률 데이터를 전처리 합니다.
# df.filter(regex=) 로 배당수익률이 들어가는 컬럼만 찾고 종목코드를 추가 합니다.
# df_income

In [ ]:
# 추후 병합을 위해 데이터프레임에 종목코드를 추가합니다.


In [ ]:
# melt 로 여러 컬럼의 종목코드를 행으로 녹이고 결측치를 dropna 로 제거합니다.
# df_in

In [ ]:
# 컬럼명을 "종목코드", "배당수익률기간", "배당수익률" 로 변경합니다.


In [ ]:
# 배당수익률% 컬럼을 만들어 전처리 합니다.



In [ ]:
# 종목코드가 종목당 2개 이상 있는 값이 있는지 확인합니다.
# 2개 이상 있다면 merge 에서 어떤 값을 사용할지 고려해야 합니다.


In [ ]:
# 전처리가 끝난 배당수익률 컬럼은 제거합니다.


## merge 로 기존 데이터와 병합하기

<img src="https://pandas.pydata.org/docs/_images/08_merge_left.svg">

* concat과 merge의 차이
    * concat :  
        * axis=0 행을 기준으로 위아래로 **같은 컬럼끼리** 값을 이어 붙여 **새로운 행을 만듦**
        * axis=1 컬럼을 기준으로 **인덱스가 같은 값**을 옆으로 붙여 **새로운 컬럼을 만듦**
    * merge :
        * index 혹은 특정 컬럼 값을 기준으로 두 개의 데이터프레임을 연결
        
<img src="https://www.dofactory.com/img/sql/sql-joins.png">

* 이미지 출처 : https://www.dofactory.com/sql/join

In [ ]:
# merge로 병합합니다.
# df와 df_pe를 병합니다. merge 전후 행과 열의 수를 확인해서 제대로 병합이 되었는지 확인합니다.
# df_merge_pe 에 할당합니다.
# df_merge_pe

In [ ]:
# 위에서 병합한 데이터(df_merge_pe)에 PBRlBPS(df_pb) 데이터도 병합해 줍니다.
# df_merge_pb

In [ ]:
# 위에서 병합한 데이터(df_merge_pb)에 배당수익률(df_in) 데이터도 병합해 줍니다.
# df_final 이라는 변수에 담습니다.
# df_final

## 사용하지 않는 컬럼 제거하기 drop

In [ ]:
# l 구분자가 들어가는 컬럼명만 찾습니다.
# del_cols

In [ ]:
# 위에서 찾은 컬럼명을 drop으로 삭제합니다.


In [ ]:
# 결측치수가 많거나 중복된 컬럼이 있는지 확인합니다.


In [ ]:
# 전처리한 컬럼은 원본 컬럼을 제거합니다.
del_cols2 = ['종목명_전처리', '시가총액', '시가총액순위', '배당수익률기간']

In [ ]:
# del_cols2 를 제거합니다.


In [ ]:
# 컬럼명 중 필요없는 컬럼이 없는지 확인합니다.


## CSV 파일로 저장하기

In [ ]:
# 기존 파일명을 변경해서 새로 저장할 파일명을 만듭니다.
file_name

In [ ]:
# 저장할 파일명을 만들어 줍니다.
# 기존 파일명에 _ver2 를 붙여줍니다.
# final_file_name

In [ ]:
# final_file_name 변수에 있는 파일명으로 저장합니다.


In [ ]:
# final_file_name 변수에 있는 값을 읽어옵니다.
# 이 때, 종목코드는 dtype={"종목코드":np.object} 로 불러옵니다.
